In [1]:
import carla
import pandas as pd
import numpy as np
import os
import csv
import sys
import importlib
import json

# Add the "scripts" directory to sys.path
current_dir = os.path.dirname(os.path.abspath("__file__"))
scripts_dir = os.path.abspath(os.path.join(current_dir, 'scripts'))
sys.path.append(scripts_dir)
import LLAMAconnect as llama

### This notebook is used to help find spawnpoints for each place of interest in the map

In [ ]:
# Connecting to CARLA
client = carla.Client('localhost', 2000)
client.set_timeout(20)
town = 'Town01'
world = client.load_world(f'{town}_Opt') # Town01_Opt is a lighter version of Town01 with layers
world.unload_map_layer(carla.MapLayer.Foliage) # unload foliage for better perfomance 
world.unload_map_layer(carla.MapLayer.Particles) # unload particles for better performance
spectator = world.get_spectator()

In [ ]:
def get_closest_spawn_point(transform, spawn_points):
    closest_point = min(spawn_points, key=lambda sp: sp.location.distance(transform.location))
    return closest_point

In [4]:
def draw_all_spawnpoints(spawn_points):
    for i, sp in enumerate(spawn_points):
        world.debug.draw_string(
            sp.location,
            f'O{i}',  # Optional: add index label
            draw_shadow=False,
            color=carla.Color(r=255, g=0, b=0),  # Red
            life_time=500.0,
            persistent_lines=True
     )

In [6]:
def draw_interest_point(spawn_points, interest, nome):
    for i, sp in enumerate(spawn_points):
        if sp.location == interest.location:
            world.debug.draw_string(
                sp.location,
                f'{nome.upper()}',  # Optional: add index label
                draw_shadow=False,
                color=carla.Color(r=0, g=255, b=0),  # Green
                life_time=500.0,
                persistent_lines=True
        )

In [7]:
spawn_points = world.get_map().get_spawn_points()
interests_spawn_points = {}
draw_all_spawnpoints(spawn_points)

In [ ]:
# Run this to create a new interest point in the map
nome = 'gym'
interests_spawn_points[nome] = get_closest_spawn_point(spectator.get_transform(), spawn_points)
draw_interest_point(spawn_points, interests_spawn_points[nome], nome)

loc = interests_spawn_points[nome].location
csv_name = f'interests_spawn_points_{town}.csv'
if os.path.exists(csv_name):
    with open(csv_name, mode='a') as file:
        writer = csv.writer(file)
        writer.writerow([nome, loc.x, loc.y, loc.z])
else:
    with open(csv_name, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Name', 'x', 'y', 'z'])
        writer.writerow([nome, loc.x, loc.y, loc.z])

In [30]:
draw_interest_point(spawn_points, interests_spawn_points[nome], nome)

In [ ]:
def get_spawn_points_from_csv(data, spawn_points):
    sps = {}
    for i in range(len(data)):
        nome = data.index[i]
        x, y, z = data.iloc[i]
        transform = carla.Transform(carla.Location(x=x, y=y, z=z))
        sps[nome] = get_closest_spawn_point(transform, spawn_points)
    return sps

In [42]:
data = pd.read_csv('interests_spawn_points.csv', index_col=0)
sps = get_spawn_points_from_csv(data, spawn_points)
places = list(sps.keys())
places

['supermarket', 'work', 'school', 'library', 'cafe', 'home', 'gym']

In [43]:
importlib.reload(llama)

response = llama.getResponse_trip(places)
while not llama.responseCheck(response, places):
    response = llama.getResponse_trip(places)

response = json.loads(response)
print(response)

{'7': {'location': 'home', 'activity': 'wake up'}, '8': {'location': 'school', 'activity': 'study'}, '9': {'location': 'school', 'activity': 'study'}, '10': {'location': 'school', 'activity': 'study'}, '11': {'location': 'school', 'activity': 'study'}, '12': {'location': 'cafe', 'activity': 'lunch'}, '13': {'location': 'library', 'activity': 'study'}, '14': {'location': 'library', 'activity': 'study'}, '15': {'location': 'library', 'activity': 'study'}, '16': {'location': 'gym', 'activity': 'practice sports'}, '17': {'location': 'library', 'activity': 'study'}, '18': {'location': 'cafe', 'activity': 'dinner'}, '19': {'location': 'library', 'activity': 'study'}, '20': {'location': 'library', 'activity': 'study'}, '21': {'location': 'home', 'activity': 'relaxation'}, '22': {'location': 'home', 'activity': 'relaxation'}, '23': {'location': 'home', 'activity': 'sleep'}}


In [ ]:
sps_routine = []
for locs in response.values():
    local = locs['location']
    sps_routine.append(sps[local])